In [1]:
import numpy as np
import torch
import scipy
import csv
from PIL import Image
import glob
import os
import imageio
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import csv
import ipdb
import sys
import base64
import random
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg

In [2]:
airfoil_file=[os.path.basename(x) for x in glob.glob('./data_geometry/*.txt')]
airfoil_list =[i.rsplit('.',1)[0] for i in airfoil_file]
my_dpi=288

for file in airfoil_list:
    data = np.loadtxt('./data_geometry/%s.txt' % (file),skiprows = 1)
    data = data.T
     
    #looping thoruhg different angles
    for alpha in range(-20,21):
        theta = alpha*np.pi/180
        
        #changing the origin to (0.5,0)
        data_R = data - scipy.array([[0.5],[0]])
        
        #rotation matrix
        R = scipy.array([[np.cos(-theta),-np.sin(-theta)],
                         [np.sin(-theta),np.cos(-theta)]])
        data1 = R @ data_R
        
        #changing origin back to (0,0) and extracting X and Y
        data1 = data1 + scipy.array([[0.5],[0]])
        X = data1[0,:]
        Y = data1[1,:] 
        
        
        plt.figure(figsize = (64/my_dpi,64/my_dpi), dpi=my_dpi)
        plt.fill(X,Y)
        plt.axis('off')
        plt.xlim(-0.1,1.1)
        plt.ylim(-0.5,0.5)
        plt.gca().set_aspect('equal')
        plt.savefig('./data_images/%s_%s.png' % (file,alpha),dpi=my_dpi)
        plt.close()
        
        image = Image.open('./data_images/%s_%s.png' % (file,alpha)).convert('L')
        image.save('./data_images/%s_%s.png' % (file,alpha))

In [3]:
re_key= np.linspace(5,757,17)
re_dict={}

lines_index = list()
for key in re_key:
    for i in range(41):
        lines_index.append(key+i)

for idx,key in enumerate(re_key):
    re_dict[key]=30000+ idx*400000 
    
airfoil_file=[os.path.basename(x) for x in glob.glob('./data_RAW/*.txt')]
airfoil_list =[i.rsplit('.',1)[0] for i in airfoil_file]

name = list()
re_list = list()
alpha_list = list()
cl_list = list()
cd_list = list()

for file in airfoil_list:
    data = open('./data_RAW/%s.txt' % file)
    k = 0
    for idx,line in enumerate(data):
        if idx in lines_index:
            if k == 41:
                k = 0
                
            wordlist=line.split()

            name.append('NACA' + file)
            re_list.append(re_dict[idx-k])
            alpha_list.append(wordlist[0])
            cl_list.append(wordlist[1])
            cd_list.append(wordlist[2])
            k +=1
            
final_data = np.column_stack((np.array(name),np.array(re_list),np.array(alpha_list),np.array(cl_list),np.array(cd_list)))

np.save("data",final_data)